# Sentiment 

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import re
import os
import json
import matplotlib.pyplot as plt
from ast import literal_eval
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [1]:
path = "C:/MyPath/"

In [20]:
analyzer = SentimentIntensityAnalyzer()

In [ ]:
df = pd.read_csv( 'DebateViccines_replies.csv')

In [31]:
def sentiment_data(df):
    analyzer = SentimentIntensityAnalyzer()
    scores = []
    # Declare variables for scores*
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []

    for i in range(df['text_cleaned'].shape[0]):
        #print(analyser.polarity_scores(sentiments_pd['text'][i]))*
        compound = analyzer.polarity_scores(df['text_cleaned'][i])["compound"]
        pos = analyzer.polarity_scores(df['text_cleaned'][i])["pos"]
        neu = analyzer.polarity_scores(df['text_cleaned'][i])["neu"]
        neg = analyzer.polarity_scores(df['text_cleaned'][i])["neg"]

        scores.append({"Compound": compound,
                       "Positive": pos,
                       "Negative": neg,
                       "Neutral": neu
                         })
        
    sentiments_score = pd.DataFrame.from_dict(scores)
    df = df.join(sentiments_score)
    return df
    
    

In [32]:
df = sentiment_data(df)

In [33]:
df

,ID,Author,Text,parent_id,Reply_to,text_cleaned,Compound,Positive,Negative,Neutral
0,qtzfiz,Overhere5150,Professional mountain biker Kyle Warner took P...,qtzfiz,['Overhere5150'],professional mountain biker kyle warner took p...,0.0000,0.000,0.000,1.000
1,t8xoe5,thekill3rpeach,Interesting development nan,t8xoe5,['thekill3rpeach'],interesting development nan,0.4019,0.574,0.000,0.426
2,s7u1t2,leviforoffice,I don't live in the uk but I wanna give you gu...,s7u1t2,['leviforoffice'],i don't live in the uk but i wanna give you gu...,-0.4199,0.185,0.236,0.579
3,su4kni,NaN,My doctor told me not to get the covid vax I w...,su4kni,[nan],my doctor told me not to get the covid vax i w...,0.9550,0.131,0.044,0.825
4,se0zmb,dionesian,"Japan bans vaccine mandates, says “do not disc...",se0zmb,['dionesian'],"japan bans vaccine mandates, says “do not disc...",0.0000,0.000,0.000,1.000
...,...,...,...,...,...,...,...,...,...,...
147673,iidd30j,PokerQuilter,Nuremberg Code...what is this you speak of?🙄. ...,iicsdoq,['hblok'],nuremberg code what is this you speak of?🙄. as...,0.6232,0.126,0.039,0.835
147674,iigcwu2,InfowarriorKat,I know what you mean. If more people would hav...,iig6hnv,['Fun_Journalist_727'],i know what you mean. if more people would hav...,-0.7509,0.073,0.129,0.798
147675,iikw82k,Mean-Copy,Exactly. It’s mind boggling how they can’t th...,iig6hnv,['Fun_Journalist_727'],exactly. it’s mind boggling how they can’t the...,0.0000,0.000,0.000,1.000
147676,iidenqc,hblok,Right. The WMA Declaration of Helsinki has sup...,iidd30j,['PokerQuilter'],right. the wma declaration of helsinki has sup...,0.0616,0.051,0.044,0.905


In [34]:
#df.to_csv( savepath + 'vax_reddit_sentiment.csv', index=False)

In [ ]:
new_max = 30
new_min = -30
max_val= df['Compound'].max()
min_val= df['Compound'].min()
old_range = max_val - min_val
new_range = new_max - new_min
df['Scaled']= [(((old_value - min_val)*new_range)/old_range)+new_min for old_value in df['Compound'] ]

In [ ]:
sentiscore= df[['Author', 'Scaled']].groupby('Author').mean().round().reset_index()
sentiscore

In [ ]:
gpath = "C:/MyPath/Graphs/"

In [ ]:
G= nx.read_weighted_edgelist(gpath + 'vax_reddit_edgelist.txt', delimiter=",")

In [ ]:
for node in G.nodes(data=True):
    if node[0] in sentiscore['Author'].tolist():
        G.nodes[node[0]]['sentiment'] = sentiscore[sentiscore['Author'] == node[0] ]['Scaled'].item()
    else:
        G.nodes[node[0]]['sentiment'] = 0

In [ ]:
for edge in G.edges(data=True):
    sentiment_diff = 60 - abs(G.nodes[edge[0]]['sentiment'] - G.nodes[edge[1]]['sentiment'])
       
    if G[edge[0]][edge[1]]['weight'] + sentiment_diff >= 1:
        G[edge[0]][edge[1]]['weight'] = G[edge[0]][edge[1]]['weight'] + sentiment_diff
    else:
        G[edge[0]][edge[1]]['weight'] = 1


In [ ]:
sentipath= "C:/MyPath/SentimentGraphs/"

In [ ]:
#save new graphs with sentiment modeling
nx.write_weighted_edgelist(G, sentipath+ 'vax_reddit_sentigraph.txt', delimiter=',')